<div class="pagebreak"></div>

# Validation, Exceptions, and Error Handling
*aka Developing Robust Code*



So far in our code, we have followed the "happy" path where everything just works and we assume our input values are correct.  If something went wrong, we either re-ran our code (e.g., bad input value from the user) or made a code fix and then re-ran the code.

Software needs to be reliable and robust. That means that it needs to prevent errors when possible, detect situations when errors do occur, and then recover from errors when possible.  

Throughout these notebooks, we have seen many errors and their associated messages - sometimes intentionally, sometimes accidentally, and sometimes just incorrect logic (semantic errors).

Now we need to make a few decisions:
1. When do we want to validate input? 
2. How do we want to validate input?
3. How do we want to prevent errors?
4. How do we want to react and recover from errors?

No clear, fixed solutions exist for these questions.  As with many things with system design, the answers will depend upon the context.

For scripts that we write for ourselves, ignoring errors may be fine in some circumstances.  However, for code or results used by others, we do need to ensure our code performs in a robust manner.  

Input validation depends upon the source, potential harm of using the data unchecked, and the ability for the exception handling to detect and handle the message.  Input validation, while often times annoying and time-consuming, is the best way to produce robust code that has a minimal amount of security flaws.  

Any data received from external sources should be be validated. This includes checking -
- data falls within allowable(expected) ranges
- numeric values can be represented within the system. While Python does not have limits on the size of integers, the language does for floats.  If you are using a database system to persist data, most likely that system will have more stringent limits. 
- for strings, we need to look at their length as well as any patterns that data might represent (e.g., Visa and Master credit cards are expected to be in the format of NNNN NNNN NNNN NNNN where N is a digit from 0 to 9.)
- malformed input designed to perform an injection attack when the data is used without proper validation or escaping [see html escape](https://docs.python.org/3/library/html.html).

You should consider checking the values of input parameters to functions and methods.

<div style="border: 3px solid black;padding: 10px; border-radius: 10px;">
    <b>Software Security Flaws</b><br>
    The <a href="https://owasp.org">Open Web Application Security Project</a> has run an awareness project over the past 
    20 years to identify the top security issues facing developers in web applications.  Looking through the
    <a href='https://web.archive.org/web/20220511193851/https://www.hahwul.com/cullinan/history-of-owasp-top-10/'>
        history</a>
    of these categories, many are directly related to the lack of input validation or sanitization (process of removing illegal characters from input or replacing potential dangerous character sequences
    with safe ones): Buffer Overflow, Cross Site Request Forgery (CSRF), Cross Site Scripting (XSS), 
    Injection, Injection Flaws, Server-Side Request Forgery, Unvalidated Input, and Unvalidated Redirects.
    Most of these now have been put into the "injection" category.
    <p> Two potentially dangerous built-in functions in Python are <code>eval()</code> and <code>exec()</code>.
        <code>eval()</code> evaluates a string assuming it is an expression. Remember - a function call is an
        expression!. <code>exec()</code> executes the contents of the string as if it represents one or 
        more Python statements.  While legitimate use cases exist for both of theses functions, extreme care
        taken to ensure any string values passed to these functions are safe to execute.
    <p>As an example, create a code cell and execute the following:
    <pre>eval('exec("import os; print(os.listdir(\'.\'))")')
    </pre>
    Listing a directory's contents seems innocuous, but the results could give valuable information to an attacker.  And if
    someone could execute that code, they more than likely could execute far more malicious code.
</div>

<p>
As we detect errors, we also need to determine who is responsible for recovering from the error as well as how to react and recover.  Is it performed within the current routine / function?  Or passed back up the caller stack?  How is the user informed?  How does this differ among programs deployed as command-line tools, local GUIs, and web applications?

A phrase long used in computer science has been "garbage in, garbage out".  The quality of work that can be performed (i.e., the results or output) is directly related to the quality of input. Use unvalidated data at your own peril.

## Revisiting User Input
In this below code snippet from the [Iteration Notebook](11-Iteration.ipynb), the user is asked to enter grades until they are complete with a negative number.  We had already added some error checking to see if the user entered at least one grade before calculating the average - this prevented a division by zero.  However, what occurs if they enter a value that's not an integer?  Let's try ...

In [ ]:
total = 0
num_entries = 0

while True:
    grade = int(input("Enter a grade: "))
    if grade < 0:
        break
    total += grade
    num_entries += 1

if num_entries > 0:
    print("Average:",total/num_entries)
else:
    print("no grades entered")

Whether you entered a string literal or a float literal, you should have received a "ValueError" that occurs when Python attempts to convert the string return value from input into an integer.  

Another error that could have occurred if we ran this script from a shell session is an "EOFError" if the input stream was closed (e.g., the user typed ctrl+d). Within Jupyter, it is not possible to replicate this issue - the environment does not provide any mechanism to close the input stream.

Several possibilities exist to validate that a string actually does represent an integer.

One possibility is to create some custom logic to ensure that each character in the string is a valid digit between 0 and 9. This option needs a check that leading character could be a negative sign. With  this approach, we would want to create a function such that other parts of our code (or even other programs) could re-use this logic.

When possible, though, we should try to reuse code. Are there any methods in Python's [string class](https://docs.python.org/3/library/stdtypes.html#string-methods)?  Looking at that documentation, several possibilities may exist: `isdecimal()`, `isdigit()`, and `isnumerical()`.

`isdecimal()` returns true as long as the string is composed of any character in the ['Unicode General Category ND' ](https://www.fileformat.info/info/unicode/category/Nd/list.htm).

In [ ]:
test_strings = ['65536','00123','-2','0.124','life42','42life','\u00BD','\u1C43','\u2460']
for s in test_strings:
    print("{:>10}".format(s), s.isdecimal(),)

Overall, this method does pretty well - although it can't handle a negative number.  We could still deal with that, though, by stripping the leading `-`.

`isdigit()` still doesn't handle negative numbers.  It also accepts numbers that are not base 10 such as ①.

In [ ]:
for s in test_strings:
    print("{:>10}".format(s), s.isdigit())

In [ ]:
int('\u2460')

`isnumeric()` is even further away from the right solution. The function accepts just about anything that can represent a number - including fractions.  And, no, it does not handle negative numbers. 

In [ ]:
for s in test_strings:
    print("{:>10}".format(s), s.isnumeric())

Another possibility covered in a later notebook is to use a regular expression. Through a powerful notational grammar, regular expressions can find both simple and complex patterns in text.

For this situation (to test if a string can be converted to an integer), we will use the regular expression `^[-+]?[\d]+$`. Here's a brief description as to how the expression works:
- `^` means to match the start of the string
- `[+-]` is a character class consisting of either the plus `+` sign or the minus `-` sign.
- `?` makes that previous character optional
- `[\d]` matches any numeric decimal character (similar to `isdecimal()`.  This includes the characters from 0 to 9 as well as those characters for other language writing systems.
- `+` means that previous character (or any member of the character class [0-9] must be present one or more times. i.e., at least once 
- `$` means to match the end of the string

From this regular expression, we can see when we convert a string to an integer, we can have an arbitrary number of leading zeros. As demonstrated earlier the `int()` function can parse leading zeros while the Python interpreter cannot parse integer literals (values embedded in th program) with leading zeros.

In [ ]:
import re
for s in test_strings:
    print("{:>10}".format(s), bool(re.match(r"^[-+]?[\d]+$",s)))

The final solution to examine goes back to that pesky `ValueError`.  Fortunately, Python allows use to capture and handle these types of errors.

## Exceptions
An exception is an error that occurs as a program executes, causing the normal execution sequence to stop processing and for control to pass to the nearest block designated to handle that type of an error. By default, if no such handle is present, the Python interpreter will print a traceback (stack trace) and stop the program.  

Another way to define an exception is an error that prevents a program from continuing normal execution.

### Handling Errors
Python provides the `try except` statement to handle errors.  The `try` block is used to contain code in which an error may occur.  The `except` block provides the necessary error handling.  You may still need more code outside of the except block to appropriately recover from the error.

In [ ]:
try:
    s = "hello"
    i = int(s)
except:
    print("'{:s}' is not a valid number.".format(s))

With no other details on the `except` line, that `except` block is a catch all for any error type.

For the grade average example, we need to determine what type of action to take when the user enters a string that is not an integer.  Ideally, we want the user to be able to recover from any inadvertent mistakes.  We also need a mechanism to allow the user to indicate they are done entering data - typically accomplished through a [sentinel value](https://en.wikipedia.org/wiki/Sentinel_value).  Our initial sentinel value was any negative number, but was that the most appropriate choice?

Rather than starting from our existing, let's revisit the pseudocode for this process:
<pre><i>
    set running total to 0
    set number of grade entries to 0
    while the user has more grade entries:
        read next grade
        add grade to running total
        increment number of entries by 1
    if there's at least one grade entry
        compute and display average
    otherwise
        state no grades entered
</i></pre>
For most of this we have a straight-forward conversion, except for the loop.  Let's expand that -
<pre><i>
    ...
    while true:
        read next grade
        if sentinel entered, exit loop
        if valid grade (an integer)
            add grade to running total
            increment number of entries by 1
        else 
            display error message
    ...
</i></pre>

Now, convert the pseudocode to Pythoncode.  Since we are changing our logic, rather than relying upon a negative integer for the sentinel, let's use the string "q"

In [ ]:
total = 0
num_entries = 0

while True:
    grade = input("Enter a grade('q to quit'): ")
    if grade == 'q':
        break
    try:
        grade = int(grade)
        total += grade
        num_entries += 1
    except:
        print("Invalid grade entered: ",grade)

if num_entries > 0:
    print("Average:",total/num_entries)
else:
    print("no grades entered")

### Handle by Type

The `except` clause with just the keyword itself catches all errors by default.  In certain circumstances, our code may only want to handle a specific error, making it the caller's responsibility to handle (or not!) any other others that may exist.

For example, what if our grades where contained in a file, one per line and we had this code to compute averages:
<pre>
with open("test.txt") as f:
    total = 0
    num_entries = 0
    for line in f:
        total += int(line)
        num_entries += 1
    print(total/num_entries)
</pre>

What errors can possibly occur?
- File does not exist
- File uses a different encoding
- File contains data other than integers
- File is empty

Now, we need to determine how to handle these different errors.

Python provides the capability to handle a specific error by specifying the specific error type after the `except`.  [View Python's Built-in Exceptions](https://docs.python.org/3/library/exceptions.html)

As you can see below, Python allows for multiple `except` blocks for each try.  Once a specific error has been handled for a `try`, the remaining `except` blocks are skipped. Because of this logic, it is also necessary to list `except` blocks in order of most specific to most generally.

Python allows us to specify a variable name after the exception. By being able to reference the exception object, we can query the exception's state (it is just an object after all) to get additional details.

The following code block contains several errors.  While the comments provide the corrections, you should run the code first.  Then fix the associated error. Repeat until the average is printed.   Finally, remove the inner most try/except handling to see how the code reacts.  What occurred?

In [ ]:
filename = "data/grade.txt"       # Correct filename is data/grades.txt
try:
    with open(file_name) as f:    # Variable name has not been defined
        total = 0
        num_entries = 0
        line_count = 0
        for line in f:
            line_count += 1
            try:
                total += int(line)
                num_entries += 1
            except ValueError:
                print("Bad integer value on line {:d}: {:s}".format(line_count,line))
        print("Average: {:.2f}".format(total/num_entries))
except FileNotFoundError as file_error:
    # show additional attributes for "FileNotFoundError"
    print(file_error.filename)
    print(file_error.errno)
    print(file_error.strerror)
    print("Unable to find:", filename)
except Exception as err:
    print("Unknown error:", err)    # this prints the excpetion's default error message
    print("Error Type:", type(err))

Python allows for multiple exceptions to be handled with the same except block:
<pre>
filename = "data/grade.txt"       # Correct filename is data/grades.txt
try:
    ...
except (FileNotFoundError, IOError) as os_error:
    # Handle OS related errors in this block
    ...
except Exception as err:
    # Handle other errors in this block
    ...
</pre>

### `else` clause
The `try`/`except` statement can have an optional `else` clause which only executes if no exceptions were raised:

<pre>
try:
    <i>statement</i>
    <i>statement</i>
    <i>...</i>
except <i>ExceptionName</i>:
    <i>statement</i>
    <i>statement</i>
    <i>...</i>
else:
    <i>statement</i>
    <i>statement</i>
    <i>...</i>
</pre>

The `else` clause must come after the `except` clause(s).

### `finally` clause
Additionally, the `try`/`except` statement can have an optional `finally` clause which excecutes the try block, the except block(s), and any `else` block. I.e., the `finally` clause executes as the last item of a `try`/`except` statement.  The purpose is to perform any necessary cleanup operations.  Code in the `finally` clause will always execute.

<pre>
try:
    <i>statement</i>
    <i>statement</i>
    <i>...</i>
except <i>ExceptionName</i>:
    <i>statement</i>
    <i>statement</i>
    <i>...</i>
finally:
    <i>statement</i>
    <i>statement</i>
    <i>...</i>
</pre>

### Raising Exceptions
With Python, we can force a specified exception to occur through the `raise` statement:

In [ ]:
raise NameError("code initiated exception")

We can catch these exception ourselves as well as re-raise them (any type of an exception can be re-raised).  The re-raise always us to perform some error handling and then delegate the rest to the caller.

In [ ]:
try:
    raise NameError("code initiated exception")
except NameError as ne:
    print("Our raised exception:",ne)
    raise ne

### Creating our Own Exceptions
While Python has a number of [built-in exception types](https://docs.python.org/3/library/exceptions.html) that we can use for our own purposes, we can also define our own exception types that are more specific for the situations that may arise in our programs. 

The built-in exceptions are classes within Python - to define our our own, we need to define a class that inherits from one of those built-in exception types.  (We'll cover the details of classes and inherits in a few notebooks.)

The follow code creates a new exception type called `MyException`.  The `pass` statement implies that the class is empty and will use the state and behavior of the parent class (`Exception`).  

In [ ]:
class MyException(Exception):
    pass

Based upon some condition in our program, we could then raise and catch this new exception type. Note: The code does not have to immediately catch the exception.  This example just catches `MyException` immediately to demonstrate the capability.

In [ ]:
try:
    raise MyException('panic')
except MyException as exc:
    print(exc)

### Stack Traces
By default, when an exception is not caught, Python produces the stack trace (Traceback) of the function/methods calls that led to the issue. It is also possible to programmatically process stack traces in code.

When the following code executes:
<pre>
def a():
    print ("called")
    raise Exception("Showing something that happened")
    
def b():
    # some processing
    a()
    # more processing
    
print("starting")
b()
print("complete")
</pre>
The following result is produced:
<pre>
starting
called
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
Input In [14], in &lt;cell line: 11&gt;()
      8     # more processing
     10 print("starting")
---> 11 b()
     12 print("complete")

Input In [14], in b()
      5 def b():
      6     # some processing
----> 7     a()

Input In [14], in a()
      1 def a():
      2     print ("called")
----> 3     raise Exception("Showing something that happened")

Exception: Showing something that happened
</pre>

The very bottom of the code shows the line where an exception occurs.  We can see the function name `a` and then the lines immediately before the exception.  We can then move upwards in the output to see which function called that one and so forth to the top.  Similarly, we can start at the top.  On line 11, our initial code routine called the function `b`.  Then on line 7, the function `a` was called within `b`.  Then on line 3 within `a` an exception occurred.

Being able to follow the code through these stack traces (tracebacks) is essential to being able to understand why exceptions occurred and then to develop code to prevent these issues.

### Exception Best Practices

As with any other programming technique, exceptions can be both used wisely and mis-abused.

Ideally, we should add exception handling anywhere an exception may occur.  However, we will want to catch/handle those exceptions at an appropriate location such that the condition causing the error can be corrected.  You should not just "use an exception to pass the buck".  Determine the best place to handle the error.  Sometimes this will be as close to the error as possible, other times it will be where the data was entered that eventually let to the exception. For exceptions caused by input data (e.g., a user entered the wrong file name), then the exception needs to be resolved in a way that the user can correct the issue - this may be several calls up the stack.

Also, for production code, we may want to include a default handler at the very top of our processing code to catch any errors that may happen.  This allows us to log these errors for future investigation.  Similarly, in processing loops for web applications, we may want to have a handle for an entry points. This allows us to provide a more appropriate error message to the user as well as to appropriately log unhandled exceptions.

Exceptions can notify other parts of the program about errors that should not be ignored.

Throw exceptions only for conditions that are truly exceptional.  If some other coding practice (validating input) can detect the condition, use that.  For example, if we are computing the average of numbers in a list, we can check that the list is not empty to prevent the division by zero error.

When creating the exception, include all of the necessary details.  For index out of bounds with a list, you could document what variable held the list, the current size, and the requested index that was out of bounds.  By having all of these details, programmerss can more easily debug the situation to determine what went wrong and then create more robust code to prevent the situation in the future.

Avoid empty `except` blocks.  If the situation can handle the exception without any special processing, then you should probably log the error message and then provide a comments as to why no error handling was present.

## Error Handling Techniques
This section summarizes many different techniques to handle errors. The right approach differs based upon the current situation.  It may be appropriate to use several of these techniques in combination. [1]

Whatever mechanism(s) utilized, document the error handling approach within function, module, or class by placing the information into docstrings. 

### Return or use a neutral /default value
One possibility is simple to return a value that's known to be harmless or devoid of meaning.  For numbers, a numeric result might be zero.  For strings, an empty string.  For collections, return an empty collection.

Within routines, if a parameter is invalid, it may be better to default to a particular value or action.  A video game could just show a default background color and let the operation continue. 

### Substitute the next piece of data
If you processing a stream or file of data, skip invalid records and just return the next legitimate record. This will often be combined with "log the error".

### Substitute the previous data value
Again, if you are processing a stream of data, such a temperature measurement or heart rate, and you do not receive a valid value, one possibility is to return the most recent legitimate value.

However, you may also need to keep track of the number of successive errors and raise an alarm when a certain threshold has passed. 

### Substitute the closest legal value / Interpolate
In other cases when processing data, you could choose to return the closest legal value.  Or if you have a piece of missing data, interpolate between the values immediately before and after.

If you are measuring radiation levels, this may not be the best idea - [Chernobyl Disaster](https://en.wikipedia.org/wiki/Chernobyl_disaster).  The Geiger counters initially used could only report radiation levels up to a certain amount.  The values after the accident far exceeded that limit. 

### Log the error
When your program detects bad data, you could choose to log the error to a file and then continue processing.  Often, you will want to use this method in conjunction with one of the other methods listed here.

As you log data, consider any potential harm that data may cause.  Was it rejected because it could lead to an injection attack?  Could the data leak sensitive or private information?

### Return an error code
Rather than the routine trying to handle the error itself, the routine can report an error has occured and let the routine's callers determine how to best respond to the situation.  Several ways exist to perform this notification:
- return an error status as the function's return value.
- raise an exception
- set a status variable

### Call an error-processing routine/class
In this approach, error handling is centralized in a global function or class.  By centralizing, the responsibility to handle errors only exists within one location. The primary advantage is that it becomes easier to investigate issues and debug problems.  The downside to this approach is that the entire system becomes tethered to this capability.

### Display an error error message
When an error occurs, alert the user to what occurred, why it occurred, and how to proceed.

<p>
<div style="border: 3px solid black;padding: 10px; border-radius: 10px;">
<b>Error messages</b><p>
Providing well-crafted and useful error messages is essential to any modern computer system.  However, that goal is not always achieved.  For instance, Jupyter Lab produced the following error message::
<pre>
Launcher Error Cannot read properties of undefined (reading 'path')
Error Invalid response: 400 Bad Request
</pre>
Not useful to say the least. 
    
Good error messages need to be - 
- Explicit and visible. At absolute minimum, users need to aware that something has gone awry.  That email lost in the Internet?  Good luck just finding that it has disappeared ...
- Human readable. Avoid obscure codes and abbreviations.  It may be necessary to include a code to provide support, but that should come at the end of the message.
- Polite phrasing. Don't blame the user or imply that they did something wrong.
- Precise description.  What exactly was the problem? In some security situations, such details may be inappropriate to provide.
- Advice.  Provide constructive advice on how to fix the problem or proceed with the next action.  

[Guidelines for Error Messages](https://web.archive.org/web/20220616061138/https://xd.adobe.com/ideas/process/information-architecture/error-message-design-ux/)    
</div>

### Be Situational
Error handling is not a one-size fits all approach.  Choose the appropriate mechanism for the current situation.  

While this approach provides flexibility, it may create risk in systems that require certain levels of robustness or safety.

### Shut down
Depending upon the situation, the best approach may be to generate an error log message and shut down immediately.  If you have software that controls radiation doses to patients as part of a chemotherapy treatment, what should occur if an error situation arises?  It is not safe to use one of the techniques listed here for imputing data.  The only safe alternative is to shut down until the situation can be appropriately resolved.

[Therac-25](https://en.wikipedia.org/wiki/Therac-25) is an often used example for this situation.  This was a radiation therapy machine that killed several patients due to software issues.   Nancy Leveson has written an [extensive paper](resources/therac25.pdf) on this situation.


## Checking Parameter Values
At the start of these notebooks, we presented the documentation that should be produced as we create an algorithm to solve a problem:  
- Inputs to the routine
- Outputs from the routine
- Pseudocode (steps in the routine)

These inputs can be thought of as parameters to a function.  As such, we should verify that these parameters meet the expected values.  If the parameters are incorrect then we need to follow one or more of the error handling techniques.

For example in this following method to compute the average of a list of values, what issues may occur?

In [ ]:
def compute_average(l):
    return sum(l)/len(l)

What happens if the data is not numeric?  What happens if we pass an empty list?
<pre>
compute_average([100,90,80,"c"])
compute_average([])
</pre>


## Discussion
Programmers should strive to develop robust programs that can continue to operate in a safe state when issues do arise.  If an executing program back into a safe state after an error, then the program needs to safely stop itself..

Trade-offs exists with the amount of error handling code added to our programs.  In some cases, too much can be as much of a curse as too little.  However, when it comes to validating external inputs into our systems - especially in online applications - data validation is absolutely critical.

## Exercises
1. Rewrite the compute_average routine to better handle non-numeric data as well as if the empty list is not defined.  Your function docstring should describe the applied error handling approach(es).

## References
1. Steve McConnell. 2004. _Code Complete_, Second Edition. Microsoft Press, USA. [O'Reilly](https://learning.oreilly.com/library/view/code-complete-2nd/0735619670/)
